In [1]:
from fastai.vision.all import *
from fastai.distributed import *
from fastai.vision.gan import *
from fastai.callback.tracker import SaveModelCallback

from timm.models.layers import patch_embed 
from timm.models import vision_transformer
from timm.models.vision_transformer import _create_vision_transformer
from models.utils.dataLoader import GramCifarLoader
from models.utils.new_losses import CriticLoss
from models.utils.metrics import Accuracy, _Accuracy

from PIL import Image
import requests

from torch import nn
import torchvision.transforms as T

import matplotlib.pyplot as plt

In [2]:
from torch.nn.parallel import DistributedDataParallel

In [3]:
DistributedDataParallel

torch.nn.parallel.distributed.DistributedDataParallel

In [4]:
torch.cuda.set_device("cuda:3")

In [5]:
H = 32
W= 32
ps=2

transform = T.Compose([
T.Resize((H,W)),
#T.ToTensor(),
T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [6]:
vit = vision_transformer._create_vision_transformer('vit_base_patch16_224',pretrained=False, img_size=H, patch_size=ps, in_chans=3, num_classes=10)

In [13]:
path = untar_data(URLs.CIFAR)
#path = untar_data(URLs.IMAGENETTE_320)
#path = 'data/GramCifar/train/train_CustomCifarlen10000_startIdx0.tar'

transform = ([*aug_transforms(),Normalize.from_stats([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

data = DataBlock(blocks=(ImageBlock, CategoryBlock), 
                 get_items=get_image_files, 
                 splitter=RandomSplitter(),
                 get_y=parent_label,
                 item_tfms=Resize(H,W),
                 batch_tfms=transform)

dloader = data.dataloaders(path,bs=10) 

In [7]:
train_ds = torch.load('data/GramCifar/train/train_CustomCifarlen10000_startIdx0.tar')
valid_ds = torch.load('data/GramCifar/train/train_CustomCifarlen10000_startIdx0.tar')

In [8]:
train_ds[0][1][1]

6

In [9]:
dloader = GramCifarLoader(train_ds, valid_ds, 5)

In [10]:
loss = CriticLoss()

In [11]:
learner = Learner(dloader, vit, loss_func=loss, metrics=[Accuracy])

In [12]:
learner.lr_find()

5
2


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [27]:
learner.fit(50,2e-7)

epoch,train_loss,valid_loss,Accuracy,time
0,1.956229,1.917618,0.299800,14:12
1,1.830651,1.845658,0.326100,14:05
2,1.745393,1.782519,0.352800,14:05
3,1.727404,1.733429,0.371100,14:02
4,1.714553,1.680927,0.392600,14:03
5,1.584060,1.647240,0.402700,14:03
6,1.561503,1.621599,0.414400,14:02
7,1.580248,1.603666,0.424400,14:02
8,1.559446,1.566504,0.436400,14:02
9,1.521465,1.569339,0.436200,13:54
